<a href="https://colab.research.google.com/github/aumat/AutoML/blob/master/H20_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!apt-get install default-jre
!java -version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.
openjdk version "11.0.7" 2020-04-14
OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)


In [ ]:
!pip install h2o
!pip install pandas~=0.24.2

In [ ]:
import h2o

In [ ]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,55 mins 23 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_unknownUser_g3y6ko
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
from h2o.automl import H2OAutoML

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train_data1.csv to train_data1 (1).csv


In [ ]:
import pandas as pd

In [ ]:
import io
train_data1 = pd.read_csv(io.BytesIO(uploaded['train_data1.csv']))

In [ ]:
train_data1.dtypes

INCIDENT_ID          object
DATE                 object
X_1                   int64
X_2                   int64
X_3                   int64
X_4                   int64
X_5                   int64
X_6                   int64
X_7                   int64
X_8                   int64
X_9                   int64
X_10                  int64
X_11                  int64
X_12                float64
X_13                  int64
X_14                  int64
X_15                  int64
MULTIPLE_OFFENSE      int64
Month                object
Year                  int64
Day                   int64
Date01                int64
dtype: object

In [ ]:
train_data1[['MULTIPLE_OFFENSE','Month','Year','Day','X_1','X_4','X_5','X_9']] = train_data1[['MULTIPLE_OFFENSE','Month','Year','Day','X_1','X_4','X_5','X_9']].apply(lambda x: x.asfactor())

In [ ]:
train_data1.dtypes

INCIDENT_ID           object
DATE                  object
X_1                 category
X_2                    int64
X_3                    int64
X_4                 category
X_5                 category
X_6                    int64
X_7                    int64
X_8                    int64
X_9                 category
X_10                   int64
X_11                   int64
X_12                 float64
X_13                   int64
X_14                   int64
X_15                   int64
MULTIPLE_OFFENSE    category
Month               category
Year                category
Day                 category
Date01                 int64
dtype: object

In [ ]:
del train_data1['DATE'],train_data1['INCIDENT_ID']

In [ ]:
del train_data1['MULTIPLE_OFFENSE']

In [ ]:
x = pd.get_dummies(train_data1)

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.60, test_size=0.40)

In [ ]:
aml = H2OAutoML(max_models = 40, seed = 10, verbosity="info", nfolds=10)

In [ ]:
aml.train(x = X_train, y = y_train)

ValueError: ignored

In [ ]:
lb = aml.leaderboard
lb.head()
lb.head(rows=lb.nrows) # Entire leaderboard

In [ ]:
test_pred=aml.leader.predict(y_train)

In [ ]:
import sklearn.metrics
print("report", sklearn.metrics.classification_report(y_test, test_pred))

In [ ]:
test_pred.head()

In [ ]:
aml.leader.model_performance(y_test)

In [ ]:
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
#se = h2o.get_model([mid for mid in model_ids if "StackedEnsemble_AllModels" in mid][0])
#metalearner = h2o.get_model(se.metalearner()['name'])

In [ ]:

import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
sn.set(font_scale=1.4)
sn.heatmap(pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, test_pred)), annot=True, annot_kws={"size": 16}, fmt='')
plt.show()

In [ ]:

print(sklearn.metrics.precision_score(y_test, test_pred))

In [ ]:
print(sklearn.metrics.recall_score(y_test, test_pred))

0.9992982456140351


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test_data1.csv to test_data1 (1).csv


In [ ]:
import io
test_data1 = pd.read_csv(io.BytesIO(uploaded['test_data1.csv']))

In [ ]:
test_data1[['Month','Year','Day','X_1','X_4','X_5','X_9']] = test_data1[['Month','Year','Day','X_1','X_4','X_5','X_9']].apply(lambda x: x.astype('category'))

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:3848: FutureWarning: 'is_extension_type' is deprecated and will be removed in a future version.  Use 'is_extension_array_dtype' instead.
  --------


In [ ]:
test_data1.dtypes

INCIDENT_ID      object
DATE             object
X_1            category
X_2               int64
X_3               int64
X_4            category
X_5            category
X_6               int64
X_7               int64
X_8               int64
X_9            category
X_10              int64
X_11              int64
X_12            float64
X_13              int64
X_14              int64
X_15              int64
Month          category
Year           category
Day            category
Date01            int64
dtype: object

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test_data1.csv to test_data1 (2).csv


In [ ]:
import io
test_copy = pd.read_csv(io.BytesIO(uploaded['test_data1.csv']))

In [ ]:
del test_data1['DATE'],test_data1['INCIDENT_ID']

In [ ]:
test_data1.dtypes

X_1       category
X_2          int64
X_3          int64
X_4       category
X_5       category
X_6          int64
X_7          int64
X_8          int64
X_9       category
X_10         int64
X_11         int64
X_12       float64
X_13         int64
X_14         int64
X_15         int64
Month     category
Year      category
Day       category
Date01       int64
dtype: object

In [ ]:
X_new = pd.get_dummies(test_data1)

In [ ]:
predictions_new = automl.predict(X_new)

In [ ]:
##INCIDENT_ID, MULTIPLE_OFFENSE

In [ ]:
solution2 = pd.DataFrame({"INCIDENT_ID":test_copy.INCIDENT_ID,"MULTIPLE_OFFENSE": predictions_new})
#Score 99.18 rank 26

In [ ]:
solution2.coalesce(1)\
   .write\
   .format('submit1-novartis.csv')\
   .options(header='true', delimiter='\t')\
   .save('gdrive/My Drive/submit_novarits1.csv')

AttributeError: ignored

In [ ]:
!pip install pandas
solution2.to_csv("submit1.csv", index = False)
!cp submit1.csv "drive/My Drive/"

ImportError: ignored

In [ ]:
with open('/gdrive/My Drive/submit1.csv', 'w') as f:
  solution2.to_csv(f)

FileNotFoundError: ignored